# Tool Error Handling Guide

This guide provides comprehensive examples and best practices for handling errors in tools
when using AutoGen AgentChat. Proper error handling ensures that agents can gracefully
handle failures and provide meaningful feedback to users.

## Overview

When tools encounter errors during execution, AutoGen automatically:

1. **Catches exceptions** raised by tool functions
2. **Sets the `is_error=True` flag** in the `FunctionExecutionResult`
3. **Passes the error message** to the model as the tool result
4. **Allows the model** to understand what went wrong and potentially take corrective action

This automatic error handling means you can simply **raise exceptions** in your tools,
and AutoGen will handle the rest.

## Basic Error Handling

The simplest approach is to raise exceptions when errors occur:

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Set up the model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key="YOUR_API_KEY",  # Set your API key
)


async def simple_calculator(operation: str, a: float, b: float) -> str:
    """Perform basic arithmetic operations."""

    # Input validation - simply raise exceptions
    if operation not in ["add", "subtract", "multiply", "divide"]:
        raise ValueError(f"Unsupported operation: {operation}. Supported: add, subtract, multiply, divide")

    if operation == "divide" and b == 0:
        raise ZeroDivisionError("Cannot divide by zero")

    # Perform calculation
    if operation == "add":
        result = a + b
    elif operation == "subtract":
        result = a - b
    elif operation == "multiply":
        result = a * b
    elif operation == "divide":
        result = a / b

    return f"Result: {a} {operation} {b} = {result}"


# Create an agent with the calculator tool
calculator_agent = AssistantAgent(
    name="calculator",
    model_client=model_client,
    tools=[simple_calculator],
    system_message="You are a helpful calculator assistant. When calculations fail, explain the error clearly.",
)

Let's test the error handling:

In [ ]:
# Test successful calculation
result = await calculator_agent.run(task="Calculate 10 + 5")
print("=== Successful Calculation ===")
print(result.messages[-1].content)
print()

# Test division by zero error
result = await calculator_agent.run(task="Divide 10 by 0")
print("=== Division by Zero Error ===")
print(result.messages[-1].content)
print()

# Test invalid operation error
result = await calculator_agent.run(task="Calculate the square root of 16")
print("=== Invalid Operation Error ===")
print(result.messages[-1].content)

## Exception Types and Their Meanings

Using specific exception types helps models understand the nature of errors:

| Exception Type | Use Case | Example |
|----------------|----------|----------|
| `ValueError` | Invalid input or data | Invalid email format, negative age |
| `FileNotFoundError` | Missing files or resources | File doesn't exist, missing configuration |
| `ConnectionError` | Network or API issues | API unreachable, network timeout |
| `PermissionError` | Authentication/authorization | Invalid API key, insufficient permissions |
| `TimeoutError` | Service timeouts | Slow API response, database timeout |
| `RuntimeError` | General runtime issues | Service unavailable, unexpected state |

## Real-World Error Handling Examples

### File Operations Tool

In [ ]:
import os
from pathlib import Path


async def file_reader(file_path: str) -> str:
    """Read content from a file with comprehensive error handling."""

    # Input validation
    if not file_path or not file_path.strip():
        raise ValueError("File path cannot be empty")

    # Convert to Path object for better handling
    path = Path(file_path)

    # Check if file exists
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    # Check if it's actually a file (not a directory)
    if not path.is_file():
        raise ValueError(f"Path is not a file: {file_path}")

    try:
        # Attempt to read the file using aiofiles for async compatibility
        import aiofiles

        async with aiofiles.open(path, "r", encoding="utf-8") as f:
            content = await f.read()

        if not content.strip():
            return f"File '{file_path}' is empty"

        return f"Content of '{file_path}':\n{content[:500]}{'...' if len(content) > 500 else ''}"

    except PermissionError as e:
        raise PermissionError(f"Permission denied reading file: {file_path}") from e
    except UnicodeDecodeError as e:
        raise ValueError(f"File '{file_path}' contains invalid text encoding") from e
    except OSError as e:
        raise RuntimeError(f"System error reading file '{file_path}': {e}") from e


# Create file operations agent
file_agent = AssistantAgent(
    name="file_assistant",
    model_client=model_client,
    tools=[file_reader],
    system_message="You help users read files. When file operations fail, explain the issue and suggest solutions.",
)

### API Integration Tool

In [ ]:
import json
from typing import Any, Dict


async def api_client(endpoint: str, method: str = "GET", data: str = "") -> str:
    """Make API calls with robust error handling."""

    # Input validation
    if not endpoint or not endpoint.strip():
        raise ValueError("API endpoint cannot be empty")

    if method.upper() not in ["GET", "POST", "PUT", "DELETE"]:
        raise ValueError(f"Unsupported HTTP method: {method}. Supported: GET, POST, PUT, DELETE")

    # Validate JSON data for POST/PUT requests
    if method.upper() in ["POST", "PUT"] and data:
        try:
            json.loads(data)
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid JSON data: {e}") from e

    # Simulate different API scenarios
    if "notfound" in endpoint.lower():
        raise FileNotFoundError(f"API endpoint not found: {endpoint}")

    if "unauthorized" in endpoint.lower():
        raise PermissionError("API authentication failed. Please check your API key.")

    if "timeout" in endpoint.lower():
        raise TimeoutError(f"Request to {endpoint} timed out. The service may be slow or unavailable.")

    if "ratelimit" in endpoint.lower():
        raise RuntimeError("API rate limit exceeded. Please wait before making more requests.")

    if "servererror" in endpoint.lower():
        raise ConnectionError(f"Server error from {endpoint}. The service may be temporarily unavailable.")

    # Simulate successful response
    mock_response = {
        "status": "success",
        "endpoint": endpoint,
        "method": method.upper(),
        "data": json.loads(data) if data else None,
        "result": "API call completed successfully",
    }

    return f"API Response: {json.dumps(mock_response, indent=2)}"


# Create API client agent
api_agent = AssistantAgent(
    name="api_assistant",
    model_client=model_client,
    tools=[api_client],
    system_message="You help users make API calls. When API calls fail, explain the error and suggest troubleshooting steps.",
)

## Testing Error Scenarios

Let's test various error scenarios to see how the agents handle them:

In [ ]:
# Test API error scenarios
api_test_cases = [
    "Make a GET request to /api/users",  # Success
    "Call the /api/notfound endpoint",  # 404 error
    "Access /api/unauthorized",  # Auth error
    "Request /api/timeout data",  # Timeout error
    "Call /api/ratelimit endpoint",  # Rate limit error
]

print("=== API Error Handling Tests ===")
for i, task in enumerate(api_test_cases, 1):
    print(f"\n--- Test {i}: {task} ---")
    result = await api_agent.run(task=task)
    print(
        result.messages[-1].content[:200] + "..."
        if len(result.messages[-1].content) > 200
        else result.messages[-1].content
    )

## Best Practices Summary

### 1. Exception Types
Use appropriate exception types to help models understand error categories:

```python
# Good: Specific exception types
if not email_is_valid(email):
    raise ValueError(f"Invalid email format: {email}")

if api_key_expired():
    raise PermissionError("API key has expired")

# Avoid: Generic exceptions
if something_wrong:
    raise Exception("Something went wrong")  # Too vague
```

### 2. Clear Error Messages
Provide actionable error messages:

```python
# Good: Actionable message
raise ValueError("File size exceeds 10MB limit. Please use a smaller file.")

# Avoid: Vague message
raise ValueError("Invalid file")  # What's invalid about it?
```

### 3. Input Validation
Validate inputs early and provide helpful feedback:

```python
async def process_age(age: int) -> str:
    # Validate early
    if not isinstance(age, int):
        raise TypeError("Age must be an integer")
    
    if age < 0:
        raise ValueError("Age cannot be negative")
    
    if age > 150:
        raise ValueError("Age seems unrealistic (>150). Please check the input.")
    
    return f"Processing age: {age}"
```

### 4. External Service Handling
Wrap external calls with specific error handling:

```python
import requests

async def call_external_api(url: str) -> str:
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.Timeout:
        raise TimeoutError(f"Request to {url} timed out")
    except requests.exceptions.ConnectionError:
        raise ConnectionError(f"Failed to connect to {url}")
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            raise PermissionError("Authentication failed")
        elif e.response.status_code == 404:
            raise FileNotFoundError(f"Endpoint not found: {url}")
        else:
            raise ConnectionError(f"HTTP error {e.response.status_code}: {e}")
```

### 5. Don't Suppress Errors
Let exceptions bubble up rather than returning error strings:

```python
# Good: Let AutoGen handle the error
async def divide_numbers(a: float, b: float) -> str:
    if b == 0:
        raise ZeroDivisionError("Cannot divide by zero")
    return str(a / b)

# Avoid: Returning error strings
async def divide_numbers_bad(a: float, b: float) -> str:
    if b == 0:
        return "Error: Cannot divide by zero"  # Model won't know this is an error
    return str(a / b)
```

## Model-Specific Error Handling

Different model providers handle tool errors differently:

- **OpenAI Models**: Receive error information and can reason about failures
- **Anthropic Models**: Have explicit support for tool error indication via the `is_error` field
- **Other Models**: Behavior may vary, but the `is_error` field provides a standard way to indicate failures

The `is_error` field in `FunctionExecutionResult` ensures consistent error communication across all model providers.

## Conclusion

Proper error handling in tools is essential for building robust AI applications. By following these practices:

1. **Simply raise exceptions** when errors occur
2. **Use specific exception types** to categorize errors
3. **Provide clear, actionable error messages**
4. **Validate inputs early** in your tool functions
5. **Handle external dependencies** with appropriate error catching

Your agents will be able to gracefully handle failures and provide meaningful feedback to users, making your applications more reliable and user-friendly.